In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd
import shapely.geometry
import pandas as pd
import numpy as np
import gzip
import dill

def get_data(data_path):
    """Return loaded data from disk."""
    with gzip.open(data_path, 'rb') as f:
        return dill.load(f)

def write_data(data, data_path):
    '''Write data to disk'''
    with gzip.open(data_path, 'wb') as f:
        dill.dump(data, f)

In [ ]:
geo_df = get_data('data/route_gdf.pkl')
geo_df.info()

In [ ]:
lats = []
lons = []
names = []
states = []
ratings = []

for feature, name, state, rating in zip(geo_df.geometry, geo_df.route_name, geo_df.state, geo_df.user_rating):
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        names = np.append(names, [name]*len(y))
        states = np.append(states, [state]*len(y))
        ratings = np.append(ratings, [np.round(rating,0)]*len(y))
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        names = np.append(names, None)
        states = np.append(states, None)
        ratings = np.append(ratings, 0)


In [ ]:
route_marks_long = (
    pd.DataFrame.from_dict({
        'lat': lats,
        'lon': lons,
        'name': names,
        'state': states,
        'rating': pd.Series(ratings, dtype=np.dtype("int32"))
    })
)

# pd.to_numeric(route_marks_long.rating, downcast="integer")
# print(route_marks_long.rating.unique())

In [ ]:
write_data(route_marks_long,'data/route_marks_long.pkl')
route_marks_df = get_data('data/route_marks_long.pkl')

In [ ]:
valid_states = ['Alabama', 'California', 'Georgia', 'Missouri', 'Illinois', 'Ohio',
       'Kentucky', 'Colorado', 'United States', 'Indiana', 'New York',
       'Vermont', 'Texas', 'Florida', 'Minnesota', 'Virginia',
       'Oklahoma', 'Arkansas', 'Maryland', 'West Virginia',
       'Michigan', 'North Carolina', 'Oregon', 'Pennsylvania',
       'Washington', 'New Jersey', 'Alaska',
       'South Carolina', 'Utah', 'New Hampshire', 'Iowa', 'Louisiana',
       'Mississippi', 'Wisconsin',
       'South Dakota', 'Wyoming', 'Massachusetts', 'New Mexico',
       'Montana', 'Idaho', 'Nevada', 'Arizona',
       'Kansas', 'Northeast', 'Southwest', 'Golf Coast', 'Southeast',
       'Tennessee', 'Nebraska', 'Delaware', 'Pacific Coast',
       'Appalachian Mountains', 'Maine', 'Rhode Island', 'Connecticut',
       'North Dakota', 'Hawaii']
us_df = geo_df.query(f'state in {valid_states}').reset_index(drop=True)

In [ ]:
us_df.drop('rep_point',axis=1)

In [ ]:
def make_route_map(geo_df):
    scl = px.colors.sequential.Viridis_r
    n_colors = len(scl)

    fig = go.Figure()

    for row in geo_df.index:
        fig.add_trace(go.Scattergeo(
            lon = pd.Series(geo_df.iloc[row].geometry.coords.xy[0]),
            lat = pd.Series(geo_df.iloc[row].geometry.coords.xy[1]),
            mode = 'lines',
            line = dict(width = .5, color=scl[round(geo_df.iloc[row].user_rating*2)-1])
            ))

    fig.update_geos(
        visible=True, resolution=110, scope="usa",
        showcountries=False, #countrycolor="Black",
        showsubunits=True, #subunitcolor="Gray"
    )
    fig.update_layout(
        showlegend = False
    )
    fig.show()
    
make_route_map(us_df)

In [ ]:
# fig = px.line_geo(lat=lats, lon=lons)
# fig.update_geos(
#     visible=False, resolution=110, scope="usa",
#     showcountries=True, countrycolor="Black",
#     showsubunits=True, subunitcolor="Gray"
# )
# fig.update_layout(height=300, margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [ ]:
fig = px.line_mapbox(route_marks_long, 
                     lat='lat', lon='lon', 
                     hover_name='name', 
                     color='rating',
                     mapbox_style="open-street-map", zoom=2)
fig.show()